<a href="https://colab.research.google.com/github/praveenpankaj/WildFireBurntAreaMapping/blob/main/Palisades_Forest_Fire_Burn_Severity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Palisades Fire
The Palisades Fire was first reported on January 7, 2025 at 10:30AM.

**Date Started**
01/07/2025 10:30 AM

**Location:** 34.1144,-118.55334

**References:**

[1] Earth Fire Alliance https://www.earthfirealliance.org/

[2] Caldor Fire: https://en.wikipedia.org/wiki/Caldor_Fire

[3] FIRMS dataset API access: https://firms.modaps.eosdis.nasa.gov/content/academy/data_api/firms_api_use.html

[4] Available Datasets: https://docs.google.com/spreadsheets/d/1hChsmwXMYNinFXaCx4SGFEfcPZ0v7MFW4xBobAnBd5w/edit?gid=0#gid=0


## Near Real-Time Data Available from FIRMS

In [ ]:
from google.colab import userdata
MAP_KEY = userdata.get('FIRMS_key')

In [ ]:
#check number of transactions
import pandas as pd
url = 'https://firms.modaps.eosdis.nasa.gov/mapserver/mapkey_status/?MAP_KEY=' + MAP_KEY
try:
  df = pd.read_json(url,  typ='series')
  display(df)
except:
  # possible error, wrong MAP_KEY value, check for extra quotes, missing letters
  print ("There is an issue with the query. \nTry in your browser: %s" % url)

,0
transaction_limit,5000
current_transactions,0
transaction_interval,10 minutes


In [ ]:
def get_transaction_count() :
  count = 0
  try:
    df = pd.read_json(url,  typ='series')
    count = df['current_transactions']
  except:
    print ("Error in our call.")
  return count

tcount = get_transaction_count()
print ('The current FIRMS data transaction count is %i' % tcount)

The current FIRMS data transaction count is 0


In [ ]:
da_url = 'https://firms.modaps.eosdis.nasa.gov/api/data_availability/csv/' + MAP_KEY + '/all'
df = pd.read_csv(da_url)
display(df)

,data_id,min_date,max_date
0,MODIS_NRT,2024-08-01,2025-02-17
1,MODIS_SP,2000-11-01,2024-07-31
2,VIIRS_NOAA20_NRT,2019-12-04,2025-02-17
3,VIIRS_NOAA21_NRT,2024-01-17,2025-02-17
4,VIIRS_SNPP_NRT,2024-12-01,2025-02-17
5,VIIRS_SNPP_SP,2012-01-20,2024-11-30
6,LANDSAT_NRT,2022-06-20,2025-02-16
7,GOES_NRT,2022-08-09,2025-02-17
8,BA_MODIS,2000-11-01,2024-10-01


MODIS_NRT: Modis Near Real Time Data (MODIS data products available through LANCE include land surface temperature, land surface reflectance, radiances, clouds/aerosols, water vapor, active fire, snow cover, and sea ice)

LANDSAT_NRT: Landsat Near Real Time Data is available within 12 hours of acquisition. Uses are to monitor land disturbances, which can increase carbon emissions and cause environmental impacts and to detect fires and thermal anomalies

In [ ]:
!pip install geopandas --quiet

In [ ]:
area_url = 'https://firms.modaps.eosdis.nasa.gov/api/area/csv/' + MAP_KEY + '/VIIRS_NOAA20_NRT/-118.55,34.10/10/2025-01-01'
df_area = pd.read_csv(area_url, sep=',', on_bad_lines='skip', engine='python')

In [ ]:
df_area.head()

,Invalid area. Expects: [west,south,east,north].


In [ ]:
try:
    import geemap, ee
except ModuleNotFoundError:
  print("Package not installed. pip in Google Colab.")
  !pip install geemap --quiet
  import geemap, ee

In [ ]:
import pandas as pd
from shapely import wkt
import geopandas as gpd
import colorcet as cc

In [ ]:
try:
        ee.Initialize(project='ee-praveenpankaj')

except Exception as e:
        ee.Authenticate()
        ee.Initialize(project='ee-praveenpankaj')

In [ ]:
loc_coords = ee.Geometry.Point([-118.55666, 34.08939])


In [ ]:
countries = ee.FeatureCollection("FAO/GAUL/2015/level0").select('ADM0_NAME')

# Filter the feature collection to subset France.
us = countries.filter(ee.Filter.eq('ADM0_NAME', 'United States of America'))

states = ee.FeatureCollection("FAO/GAUL/2015/level1")
state_aoi = states.filter(ee.Filter.eq('ADM1_NAME', 'California'))

counties = ee.FeatureCollection("FAO/GAUL/2015/level2")
county_aoi = counties.filter(ee.Filter.eq('ADM2_NAME', 'Los Angeles'))

In [ ]:
M = geemap.Map()
M.add_basemap("HYBRID")
M.addLayer(us, {}, "USA")
M.addLayer(state_aoi, {}, 'California')
M.addLayer(county_aoi, {'color': 'red'}, 'Los Angeles County')
M.centerObject(ee_object=loc_coords, zoom=15)

#M.centerObject(ee_object = county_aoi, zoom=15)

In [ ]:
M

Map(center=[34.009409000000005, -118.31729500000003], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
countries = ee.FeatureCollection("FAO/GAUL/2015/level2")

filtered_countries = countries.filter(ee.Filter.stringStartsWith('ADM2_NAME', 'Los'))
unique_names = filtered_countries.reduceColumns(
    reducer=ee.Reducer.frequencyHistogram(),
    selectors=['ADM2_NAME']
).get('histogram').getInfo()

# Print the number of features found (optional)
print(f'Number of features with ADM_1) starting with "Los": {len(unique_names)}')

# Get the number of features
num_features = len(unique_names)

print(unique_names)

Number of features with ADM_1) starting with "Los": 30
{'Los Alamos': 1, 'Los Aldamas': 1, 'Los Amates': 1, 'Los Andes': 4, 'Los Angeles': 1, 'Los Arabos': 1, 'Los Cacaos': 1, 'Los Cedrales': 1, 'Los Chiles': 1, 'Los Cordobas': 1, 'Los Herreros': 1, 'Los Hidalgos': 1, 'Los Lagos': 1, 'Los Llanos': 1, 'Los Palacios': 1, 'Los Palmitos': 1, 'Los Patios': 1, 'Los Pozos': 1, 'Los Ramones': 1, 'Los Reyes': 2, 'Los Reyes De Juarez': 1, 'Los R¡os': 1, 'Los Salias': 1, 'Los Santos': 2, 'Los Tanques': 1, 'Losice': 1, 'Loska Dolina': 1, 'Loski Potok': 1, 'Lospalos': 1, 'Losuia': 1}


In [ ]:
# Burn Area Index Calculation
def calc_bais(image):
    return ee.Image(image.expression(
    '(1 - ((R2*R4*R4)/(R4))**(0.5))*((SWIR2 - R4)/((SWIR2 + R4)**(0.5)) + 1)', {
      'RED': image.select('B4'),
      'R2': image.select('B6'),
      'R3': image.select('B7'),
      'SWIR2': image.select('B12'),
      'R4': image.select('B8A'),
}))

**Short wave infrared (SWIR)** measurements can help scientists estimate how much water is present in plants and soil, as water absorbs SWIR wavelengths. Short wave infrared bands (a band is a region of the electromagnetic spectrum; a satellite sensor can image Earth in different bands) are also useful for distinguishing between cloud types (water clouds versus ice clouds), snow and ice, all of which appear white in visible light. In this composite vegetation appears in shades of green, soils and built-up areas are in various shades of brown, and water appears black. Newly burned land reflects strongly in SWIR bands, making them valuable for mapping fire damages. Each rock type reflects shortwave infrared light differently, making it possible to map out geology by comparing reflected SWIR light.

For Sentinel-2 SWIR composite: (B12, B8A, B04)

### Sentinel-2 data visualization

In [ ]:
def maskS2clouds(image):
    qa = image.select('QA60');

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) and qa.bitwiseAnd(cirrusBitMask).eq(0)

    return image.updateMask(mask).divide(10000)

In [ ]:
# prompt: in the code below filter imagecollection for cloud using ee.Filter.eq 'CLOUD_COVER'

# Assuming you have an ImageCollection named 'imageCollection'
# Replace 'imageCollection' with the actual name of your ImageCollection

def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) and qa.bitwiseAnd(cirrusBitMask).eq(0)

    return image.updateMask(mask).divide(10000)

# Filter the ImageCollection for cloud cover
filtered_imageCollection = imageCollection.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) # Example: Cloud cover less than 10%

#Alternatively, if you have a 'CLOUD_COVER' property:
#filtered_imageCollection = imageCollection.filter(ee.Filter.lt('CLOUD_COVER', 10))


# Now 'filtered_imageCollection' contains only images with less than 10% cloud cover.
# You can then apply other functions like map to process the images.
# For example:
# masked_collection = filtered_imageCollection.map(maskS2clouds)


In [ ]:
# Sentinel-2 image filtered
# Sentinel-2 acquisition plan: https://sentinel.esa.int/web/sentinel/missions/sentinel-2/acquisition-plans
# Predict their passes: http://www.satflare.com/track.asp?q=40697#LIST
# https://www.researchgate.net/post/How_can_I_know_when_Sentinel-2_passes_over_a_location

se2_bb = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(
    '2024-11-01', '2024-12-30').filterBounds(county_aoi)

se2_ab = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(
    '2025-01-12', '2025-02-17'). \
    filterBounds(county_aoi). \
    filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 10))

#rgb_bands = ['B4', 'B3', 'B2']
swir_rgb = ['B12', 'B8A', 'B4']

# set some thresholds
# Sentinel-2 Composites: https://github.com/sentinel-hub/custom-scripts/tree/master/sentinel-2/composites
#rgbViz = {"min":0.0, "max": 0.3, "bands": rgb_bands}
swirgbViz = {"min": 0.0, "max": 0.4, "bands": swir_rgb}

In [ ]:
M.addLayer(se2_bb.median().divide(10000).clip(county_aoi), swirgbViz, 'Before Fire')
M.addLayer(se2_ab.median().divide(10000).clip(county_aoi), swirgbViz, 'After Fire')

In [ ]:
M

Map(bottom=418933.0, center=[34.06830277687429, -118.54969024658205], controls=(WidgetControl(options=['positi…

# NBR Calculation

Reference: https://colab.research.google.com/drive/1GUbBRdlXsq1szNoX5tS_LMAHakkXNdzL?usp=sharing#scrollTo=Qb5Ekvr6jWpo

In [ ]:
bais_after = se2_ab.map(maskS2clouds).map(calc_bais)
bais_before = se2_bb.map(maskS2clouds).map(calc_bais)

In [ ]:
delta_bais = bais_after.mosaic().subtract(bais_before.mosaic())

In [ ]:
delta_bais = delta_bais.multiply(1000)

In [ ]:
thresholds = ee.Image([-1000, -251, -101, 99, 269, 439, 659, 2000]);
bais_class = delta_bais.lt(thresholds).reduce('sum').toInt()
labels = ['NA', 'High Severity', 'Moderate-high Severity',
          'Moderate-low Severity', 'Low Severity','Unburned',
          'Enhanced Regrowth, Low', 'Enhanced Regrowth, High']


In [ ]:
M.addLayer(delta_bais.clip(county_aoi),
             {'min': 0, 'max': 500, 'palette': ['red', 'black']},
              'Difference Burnt Area')
cpalette = ['7a8737', 'acbe4d', '0ae042', 'fff70b', 'ffaf38', 'ff641b', 'a41fd6', 'ffffff'];

M.addLayer(bais_class.clip(county_aoi), {'palette': cpalette}, 'Burn Severity')
M.addLayerControl()

In [ ]:
visualization = {
  'min': 100,
  'max': 700,
  'palette': cc.fire
}

In [ ]:
loc_coords.coordinates().getInfo()

[-118.55666, 34.08939]

In [ ]:
# initialize our map
Map = geemap.Map()
Map.add_basemap("SATELLITE")
Map.add_marker(loc_coords.coordinates().getInfo(), title = 'Palisades Fire Origin')
Map.addLayer(delta_bais.clip(county_aoi), visualization, 'Burnt Area')
Map.centerObject(ee_object=loc_coords, zoom=15)

In [ ]:
Map

Map(center=[34.089389999999995, -118.55666000000002], controls=(WidgetControl(options=['position', 'transparen…

The FDC algorithm provides a “fire mask code” for each pixel, with a few dozen predefined values. Here we focus on the following code values: processed fire pixel (value 10), saturated fire pixel (11), cloud contaminated fire pixel (12), high probability fire pixel (13), medium probability fire pixel (14), low probability fire pixel (15); and the corresponding “temporally filtered” code values: temporally filtered process fire pixel (30), temporally filtered saturated fire pixel (31), etc. These mask codes (10–15 and 30–35) indicate that the pixel is believed to cover a wildfire, with a varying degree of confidence.

In [ ]:
!pip install segment-geospatial pycrs

In [ ]:
from samgeo import SamGeo

In [ ]:
bbox = Map.user_roi_coords()

In [ ]:
geemap.ee_to_geotiff(
    se2_ab, "swir.tif", bbox, zoom=17, vis_params={"bands": ['B12', 'B8A', 'B4']}
    )

Downloaded image 1/21384
Downloaded image 2/21384
Downloaded image 3/21384
Downloaded image 4/21384
Downloaded image 5/21384
Downloaded image 6/21384
Downloaded image 7/21384
Downloaded image 8/21384
Downloaded image 9/21384
Downloaded image 10/21384
Downloaded image 11/21384
Downloaded image 12/21384
Downloaded image 13/21384
Downloaded image 14/21384
Downloaded image 15/21384
Downloaded image 16/21384
Downloaded image 17/21384
Downloaded image 18/21384
Downloaded image 19/21384
Downloaded image 20/21384
Downloaded image 21/21384
Downloaded image 22/21384
Downloaded image 23/21384
Downloaded image 24/21384
Downloaded image 25/21384
Downloaded image 26/21384
Downloaded image 27/21384
Downloaded image 28/21384
Downloaded image 29/21384
Downloaded image 30/21384
Downloaded image 31/21384
Downloaded image 32/21384
Downloaded image 33/21384
Downloaded image 34/21384
Downloaded image 35/21384
Downloaded image 36/21384
Downloaded image 37/21384
Downloaded image 38/21384
Downloaded image 39/2

Exception: Server error '503 Service Unavailable' for url 'https://earthengine.googleapis.com/v1/projects/ee-praveenpankaj/maps/5b9c91e7300f6e76f5ceeeee27358ce5-535bcecf0b531542b0acca9cc9646db7/tiles/17/22307/52328'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/503

In [ ]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    device=None,
    sam_kwargs=None,
)

In [ ]:
sam.generate("swir.tif", output="masks.tif", foreground=True, unique=True)

In [ ]:
sam.show_masks(cmap="binary_r")

In [ ]:
sam.show_anns(axis="off", alpha=1, output="annotations.tif")